In [2]:
import PyPDF2
import docx
import re
from typing import List, Dict
import os


In [3]:
def extract_pdf_headings(pdf_path: str) -> List[Dict[str, str]]:
    """
    Extract headings from a PDF file based on font size and bold attributes.
    Returns a list of dictionaries containing heading text and estimated level.
    """
    headings = []
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                # Extract text and formatting (simplified approach)
                text = page.extract_text()
                lines = text.split('\n')
                
                for line in lines:
                    line = line.strip()
                    if not line:
                        continue
                    # Heuristic: Headings are typically short, bold, or larger font
                    # PyPDF2 doesn't directly provide font info, so we use length and content
                    if len(line.split()) < 10 and line.isupper() or re.match(r'^\d+\.?\s', line):
                        headings.append({
                            'text': line,
                            'level': estimate_heading_level(line)
                        })
    except Exception as e:
        print(f"Error processing PDF {pdf_path}: {str(e)}")
    return headings

def extract_docx_headings(docx_path: str) -> List[Dict[str, str]]:
    """
    Extract headings from a DOCX file based on paragraph styles.
    Returns a list of dictionaries containing heading text and level.
    """
    headings = []
    try:
        doc = docx.Document(docx_path)
        for para in doc.paragraphs:
            if para.style.name.startswith('Heading'):
                level = para.style.name.replace('Heading', '').strip()
                if level.isdigit():
                    headings.append({
                        'text': para.text.strip(),
                        'level': int(level)
                    })
            elif para.text.strip() and len(para.text.split()) < 15:
                # Fallback for non-standard heading styles
                if para.runs and any(run.bold or run.font.size for run in para.runs):
                    headings.append({
                        'text': para.text.strip(),
                        'level': estimate_heading_level(para.text)
                    })
    except Exception as e:
        print(f"Error processing DOCX {docx_path}: {str(e)}")
    return headings

def estimate_heading_level(text: str) -> int:
    """
    Estimate heading level based on text characteristics (heuristic).
    Returns an integer representing the heading level (1-6).
    """
    if re.match(r'^\d+\.\s', text):  # e.g., "1. Introduction"
        return 1
    elif re.match(r'^\d+\.\d+\s', text):  # e.g., "1.1 Subheading"
        return 2
    elif text.isupper():
        return 1
    elif len(text.split()) < 5:
        return 2
    return 3  # Default level for other cases

def process_file(file_path: str) -> List[Dict[str, str]]:
    """
    Process a file (PDF or DOCX) and extract its headings.
    """
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()
    
    if ext == '.pdf':
        return extract_pdf_headings(file_path)
    elif ext == '.docx':
        return extract_docx_headings(file_path)
    else:
        raise ValueError(f"Unsupported file format: {ext}")

def main():
    # Example usage
    file_path = input("Enter the path to your PDF or DOCX file: ")
    if not os.path.exists(file_path):
        print("File not found!")
        return
    
    try:
        headings = process_file(file_path)
        print("\nExtracted Headings:")
        for heading in headings:
            print(f"Level {heading['level']}: {heading['text']}")
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()



Extracted Headings:
Level 1: UBND THÀNH PHỐ HÀ NỘI
Level 1: TRƯỜNG ĐH THỦ ĐÔ HÀ NỘI
Level 1: CỘNG HOÀ XÃ HỘI CHỦ NGHĨA VIỆT NAM
Level 1: QUYẾT ĐỊNH
Level 1: HIỆU TRƯỞNG TRƯỜNG ĐẠI HỌC THỦ ĐÔ HÀ NỘI
Level 1: QUYẾT ĐỊNH:
Level 1: UBND THÀNH PHỐ HÀ NỘI
Level 1: THỦ ĐÔ HÀ NỘI
Level 1: NHỮNG QUY  ĐỊNH CHUNG
Level 1: 1. Quy chế này quy định chung về tổ chức và quản lý đào tạo trình độ đại học  tại
Level 1: 2. Quy chế này áp dụng  trong đào tạo theo hình thức chính quy và hình thức vừa
Level 1: 3. Quy chế này là căn cứ  để xây dựng và ban hành các văn bản quy định cụ thể
Level 1: 1. Chương trình đào tạo  được xây dựng theo đơn vị tín chỉ, cấu trúc từ các môn
Level 1: 2. Nội dung , chuẩn đầu ra của  chương trình đào tạo áp dụng  chung đối với  các
Level 1: 3. Chương trình đào  tạo được công khai đối với người học  trước khi tuyển sinh
Level 1: 4. Đối với mỗi hình thức đào tạo, chương trình đà o tạo có đầy đủ nội dung nhằm
Level 1: 5. Thời gian tối đa để sinh viên  hoàn thành khoá học  không v